# imports

In [1]:
import json
import os
from tqdm import tqdm
import numpy as np

In [2]:
from concurrent.futures import ProcessPoolExecutor

In [3]:
from sentencepiece import SentencePieceProcessor

# paths

In [4]:
data = './data'

In [5]:
tokenizer_path = './tokenizer.model'
# this is the same tokenizer as found in llama2

# download dataset

In [6]:
with open(os.path.join(data, 'data07.json'), 'r') as f:
    json_data = json.load(f)

In [7]:
print(len(json_data))
json_data[:2]

100000


[{'story': "Once upon a time, there was a big, fierce lion. He lived in a jungle with lots of trees and animals. One day, the lion was walking around and he saw a lab. The lab was a big building with lots of rooms and people inside.\nThe lion thought it would be fun to hang from the lab's roof. So, he climbed up the building and hung from the edge. The people inside were surprised and scared when they saw the lion hanging from the roof!\nBut then, the lion realized he was being silly and climbed back down. He decided to go back to the jungle and play with his animal friends instead. The end.",
  'instruction': {'prompt:': 'Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand. In the story, try to at some point use the verb "hang", the noun "lab" and the adjective "fierce". Remember to only use simple words!',
   'words': ['hang', 'lab', 'fierce'],
   'features': []},
  'summary': 'A lion climbs up a lab building and hangs from 

# tokenize

In [8]:
tokenizer = SentencePieceProcessor(model_file=tokenizer_path)

In [9]:
print(tokenizer.vocab_size())
print(tokenizer.bos_id(), tokenizer.eos_id())

32000
1 2


In [10]:
story = json_data[0]['story'].strip()
tokens = [tokenizer.bos_id()] + tokenizer.encode(story)
print(len(story)), print(len(story.split())), print(len(tokens))
print(tokens[:10])

584
116
150
[1, 9038, 2501, 263, 931, 29892, 727, 471, 263, 4802]


In [11]:
def tokenize_chunk(chunk_path):
    print(chunk_path)
    with open(chunk_path, 'r') as f:
        chunk = json.load(f)
    all_tokens = []
    for sample in tqdm(chunk):
        story = sample['story'].strip()
        all_tokens.extend([tokenizer.bos_id()] + tokenizer.encode(story))
    all_tokens = np.array(all_tokens, dtype=np.uint16)
    with open(chunk_path.replace('.json', '.bin'), "wb") as f:
        f.write(all_tokens.tobytes())
    # calculate the average sequence length (they are separated by BOS=1)
    avg_seq_len = all_tokens.size / ((all_tokens == tokenizer.bos_id()).sum())
    print(f"Saved {tokenized_filename}, average seqlen: {avg_seq_len:.2f}")
        
    
def tokenize_all_chunks(data, max_workers=5):
    json_paths = [os.path.join(data, fn) for fn in os.listdir(data) if fn.endswith('json')]
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        executor.map(tokenize_chunk, json_paths)
        
        

In [12]:
tokenize_all_chunks(data)

./data/data17.json./data/data42.json
./data/data30.json
./data/data14.json./data/data11.json




 96%|████████████████████████████████████████████████████████████████████▊   | 95522/100000 [00:42<00:01, 2414.64it/s]

./data/data26.json


100%|███████████████████████████████████████████████████████████████████████▊| 99685/100000 [00:45<00:00, 1977.89it/s]

./data/data49.json
./data/data39.json


...

 35%|████████████████████████▊                                               | 34545/100000 [00:14<00:28, 2312.12it/s]

./data/data41.json


  2%|█▍                                                                       | 2053/100000 [00:00<00:45, 2153.24it/s]

./data/data16.json


100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:42<00:00, 2328.24it/s]


In [13]:
with open('./data/data01.bin', 'rb') as f:
    x = f.read()
np.frombuffer(x, dtype=np.uint16)

array([    1,  9038,  2501, ...,  1407,  9796, 29889], dtype=uint16)

In [14]:
with open('./data/data07.bin', 'rb') as f:
    x = f.read()
x = np.frombuffer(x, dtype=np.uint16)

In [15]:
x = x[:1000]

In [16]:
print(tokenizer.decode(x.tolist()))

Once upon a time, there was a big, fierce lion. He lived in a jungle with lots of trees and animals. One day, the lion was walking around and he saw a lab. The lab was a big building with lots of rooms and people inside.
The lion thought it would be fun to hang from the lab's roof. So, he climbed up the building and hung from the edge. The people inside were surprised and scared when they saw the lion hanging from the roof!
But then, the lion realized he was being silly and climbed back down. He decided to go back to the jungle and play with his animal friends instead. The end. Once upon a time there was a blue bear called Pat. Pat loved to stay in the woods, but he had no friends. One day, he heard a powerful roar coming from the cave nearby. Scared, he decided to stay away.
The next day, Pat decided to explore the cave. He wanted to know what was making the loud noise.
He went inside and saw a huge, powerful dragon. The dragon said to Pat, “You can stay and be my friend, or you can g

In [64]:
len(x)

20638398

In [65]:
x

array([    1,  9038,  2501, ...,  3926,  1156, 29889], dtype=uint16)